In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer, gen_rbf_X, create_clus_split
from spe.relaxed_lasso import RelaxedLasso
from spe.forest import BlurredForest
from spe.estimators import kfoldcv, kmeanscv, \
                           bag_kfoldcv, bag_kmeanscv, \
                           cp_relaxed_lasso_train_test, \
                           cp_bagged_train_test, \
                           cp_rf_train_test, \
                           better_test_est_split

In [2]:
err_cmp = ErrorComparer()

In [82]:
niter = 100

n=30**2
# n=20**2
p=30
s=30

# k=3
n_estimators=100
max_depth=6

In [83]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [84]:
rho = 0.5
delta = 0.8

D = np.abs(c_x - c_x[:,None]) + np.abs(c_y - c_y[:,None])
Sigma_t = rho**D
Sigma_t = delta*Sigma_t + (1-delta) * np.eye(n)

# m = 20
# c = np.zeros(n)
# c[:m] = [rho**i for i in np.arange(m)]
# Sigma_t = toeplitz(c)

# b = n / m
# c = [rho**i for i in np.arange(m)] 
# Sigma_t = block_diag(*[toeplitz(c) for _ in np.arange(b)])

Chol_t = np.linalg.cholesky(Sigma_t)
# Sigma_t = np.eye(n)
# Chol_t = np.eye(n)

In [85]:
# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

X = gen_rbf_X(c_x, c_y, p)
# X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [86]:
# xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
# pts = np.stack([xv.ravel(), yv.ravel()]).T
# n = nx*ny
# rn = int(np.sqrt(n))
# ctr = np.random.choice(pts.shape[0], size=rn, replace=True)
# ctr = pts[ctr]
# tr_idx = np.vstack([[pt + np.array((1.25*np.random.randn(2)).astype(int)) for _ in np.arange(rn)] for pt in ctr])
# tr_idx = np.maximum(0, tr_idx)
# tr_idx[:,0] = cx = np.minimum(nx-1, tr_idx[:,0])
# tr_idx[:,1] = cy = np.minimum(ny-1, tr_idx[:,1])
# tr_idx = np.unique(np.ravel_multi_index(tr_idx.T, (nx,ny)))
# tr_idx.shape

In [87]:
(test_err,
 rnd_err,
 trc_err) = err_cmp.compare(BlurredForest(n_estimators=n_estimators, max_depth=max_depth),
                                 [better_test_est_split, cp_rf_train_test, cp_rf_train_test],
                                 [{}, {'use_trace_corr': False}, {'use_trace_corr': True}],
                                 niter=niter,
                                 n=n,
                                 p=p,
                                 s=s,
                                 snr=0.4, 
                                 X=X,
                                 beta=beta,
                                 coord=coord,
                                 Chol_t=Chol_t,
                                 Chol_s=None,
                                 tr_idx=None,
                                 fair=False,
#                                  bootstrap_type='blur',
                                 )

0
0.5833333333333334
10
20
30
40
50
60
70
80
90


In [88]:
# fig_X = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=X[:,13], opacity=0.50)])
# fig_X.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Feature Xj'),
# )
# fig_X.show()

In [89]:
# fig_y = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=y, opacity=0.50)])
# fig_y.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Response Y'),
# )
# fig_y.show()

In [90]:
# fig_mu = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=mu, opacity=0.50)])
# fig_mu.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Mean'),
# )
# fig_mu.show()

In [91]:
# fig_eps = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=y-mu, opacity=0.50)])
# fig_eps.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Eps'),
# )
# fig_eps.show()

In [92]:
# groups = KMeans(n_clusters=10).fit(tr_idx).labels_

In [93]:
# px.scatter(pd.DataFrame({'x': tr_idx[:,0], 'y': tr_idx[:,1], 'g': groups}), x='x', y='y', color='g')

In [94]:
risk = test_err.mean()
risk_trc = trc_err.mean()
risk_rnd = rnd_err.mean()
# risk, risk_kfcv, risk_spcv, risk_lin

In [95]:
df = pd.DataFrame({'Rand': (rnd_err.T),# - test_err.T) / test_err.T,
                   'Trace': (trc_err.T)})# - test_err.T) / test_err.T})

In [96]:
fig_lin = px.box((df - risk) / risk, 
             labels={
                     "variable": "Method",
                     "value": "Relative MSE"
                     },
             title="lin correction",
             points=False)
fig_lin.update_traces(boxmean=True)

fig_lin.add_hline(y=0., line_color='red')

In [97]:
fig = go.Figure(data=go.Scatter(
        x=[0, 1, 2],
        y=((df - risk)/risk).mean(),
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=((df - risk)/risk).std(),
            visible=True)
    ))
fig.show()

In [98]:
(df/risk).mean()

Rand     0.893349
Trace    0.885884
dtype: float64

In [43]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['Rand', 'Trace'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title="Linear Regression Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
fig.show()

In [99]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['Rand', 'Trace'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title="Linear Regression Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
fig.show()

In [63]:
df.var()

Rand     0.031176
Trace    0.059325
dtype: float64